In [1]:
import numpy as np
import pandas as pd
import time

from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sys
import os
sys.path.insert(1, os.getcwd())

from Locators import URL, AD_CLOSE_BTN, POP_CLOSE, DRPDN_EL, TABLE_CONTENT, wait_5, wait_10
from PL_Page import PL_DATA

In [2]:
driver = webdriver.Chrome()
pl_page = PL_DATA(driver)

In [3]:
pl_page.close_ads(AD_CLOSE_BTN)
pl_page.close_ads(POP_CLOSE)

In [4]:
pl_page.click(DRPDN_EL)
gw = 1
GW_PATH = (By.XPATH, f'//*[@class="tabbedContent"]//*[@data-dropdown-block="gameweekNumbers"]//*[@data-option-name={gw}]')
pl_page.click_script(GW_PATH)
time.sleep(1)

In [5]:
table_text = pl_page.is_present_with_text(TABLE_CONTENT, 'Arsenal')
if table_text is None:
    raise(KeyboardInterrupt)
print('Dropdown option clicked!\nFetching table data ...')

table_df = pl_page.fetch_pl_table_data(TABLE_CONTENT[1], gw)
time.sleep(1)

Dropdown option clicked!
Fetching table data ...


In [7]:
for gw in range(2,39):
    pl_page.click(DRPDN_EL)
    GW_PATH = (By.XPATH, f'//*[@class="tabbedContent"]//*[@data-dropdown-block="gameweekNumbers"]//*[@data-option-name={gw}]')
    pl_page.click_script(GW_PATH)
    print(gw, end='\r')
    
    time.sleep(1)
    table_text = pl_page.is_visible(TABLE_CONTENT)
    if table_text is None:
        raise(KeyboardInterrupt)
    
    time.sleep(2)
    table_df_temp = pl_page.fetch_pl_table_data(TABLE_CONTENT[1], gw)
    table_df = pd.merge(table_df, table_df_temp, on='Teams', how='outer')

pl_page.close()

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


In [10]:
table_df

,Teams,Position-1,Goals_For-1,Goal_Against-1,Points-1,Position-2,Goals_For-2,Goal_Against-2,Points-2,Position-3,...,Goal_Against-36,Points-36,Position-37,Goals_For-37,Goal_Against-37,Points-37,Position-38,Goals_For-38,Goal_Against-38,Points-38
0,Arsenal,1,3,0,3,3,5,1,6,5,...,38,55,9,53,39,58,8,55,39,61
1,Leicester City,2,3,0,3,1,7,2,6,1,...,44,66,5,66,46,66,5,68,50,66
2,Chelsea,3,3,1,3,11,3,3,3,9,...,33,64,3,57,34,67,4,58,36,67
3,Newcastle United,4,2,0,3,13,2,3,3,10,...,62,39,15,44,62,42,12,46,62,45
4,Wolverhampton Wanderers,5,2,0,3,12,3,3,3,16,...,49,45,12,35,50,45,13,36,52,45
5,Liverpool,6,4,3,3,4,6,3,6,2,...,42,63,4,66,42,66,3,68,42,69
6,Crystal Palace,7,1,0,3,5,4,1,6,6,...,61,44,13,41,64,44,14,41,66,44
7,Everton,8,1,0,3,2,6,2,6,3,...,43,56,8,47,43,59,10,47,48,59
8,Leeds United,9,3,4,0,10,7,7,3,7,...,53,53,10,59,53,56,9,62,54,59
9,Southampton,10,0,1,0,19,2,6,0,15,...,63,43,14,47,65,43,15,47,68,43


In [11]:
table_df.to_csv('pl_fullstat_20-21.csv')

## Plotting

In [129]:
# teams to watch
clubs = {'Manchester United': ['#e60000', 1, 3], 'Manchester City': ['#33d6ff', 1, 3], 
         'Liverpool': ['#cc0000', 1, 3], 'Chelsea': ['#005ce6', 1, 3], 
         'Arsenal': ['#ff4d4d', 0.33, None], 'Leicester City': ['#668cff', 0.33, None], 
         'Everton': ['#4d4dff', 0.33, None], 'Tottenham Hotspur': ['#b1cdcd', 0.33, None]}

In [113]:
topclubs = pl_stats.loc[pl_stats['Teams'].isin(clubs.keys())]

In [ ]:
fig = go.Figure()
for club, attr in clubs.items():
    y = topclubs[topclubs['Teams'] == club].iloc[0,1:]
    color, op, lw = attr
    fig.add_trace(go.Scatter(x=topclubs.columns[1:], y=y,
                             opacity=op,
                             name=club, line=dict(width=lw, color=color)))

fig.update_yaxes(autorange='reversed', title='Position')
fig.update_xaxes(title='Game Week')
fig.update_layout(width = 1500, height = 550)